In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

In [19]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,age_55,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
count,7.603700e+04,76037.000000,76037.000000,76037.0,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,...,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000
mean,4.994795e+07,0.052225,0.467102,1.0,0.085866,0.090824,0.059405,0.096098,0.132935,0.112366,...,2.175033,0.968765,1.511600,3.703697,3.758749,2.178216,2.885066,1.580296,0.299999,0.344569
std,2.890149e+07,0.222481,1.612204,0.0,0.280168,0.287361,0.236383,0.294727,0.339507,0.315819,...,2.243436,1.046544,1.536634,4.089940,3.434990,2.533432,2.696510,1.814171,0.458260,0.475231
min,2.100000e+02,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483922e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990758e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,1.200000,2.000000,3.000000,1.000000,2.600000,1.000000,0.000000,0.000000
75%,7.510115e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.285714,7.000000,6.000000,3.000000,4.800000,2.000000,1.000000,1.000000
max,9.999863e+07,1.000000,15.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.000000,7.000000,7.000000,12.000000,10.000000,10.000000,10.000000,12.000000,1.000000,1.000000


In [20]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

X = data.loc[:, data.columns != 'TARGET']
y = data['TARGET']
y.describe()

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [21]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    pred_y = pred_y.clip(min=0)
    pred_y = pred_y.clip(max=15)
    pred_y = pred_y.astype(int)
    # The root mean square logarithmic error
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    
    print ("RMSLE: %.4f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(pred_y, y_test)
    print ("R2 value: %.4f" % r2)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=99)

# pipe = Pipeline([
#     ('scale', StandardScaler()),
#     ('regr', Lasso())
# ])

# param_grid = [
#     {
#         'regr': [Lasso(), Ridge()],
#         'regr__alpha': np.logspace(-4, 1, 6),
#     },
#     {
#         'regr': [SGDRegressor()],
#         'regr__alpha': np.logspace(-5, 0, 6),
#         'regr__max_iter': [500, 1000],
#     }
# ]

# grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
# grid.fit(X_train, y_train)

#eval(grid, X_test, y_test)grid.best_params_, grid.best_score_


In [13]:
grid.best_params_, grid.best_score_


({'regr': Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 'regr__alpha': 0.001},
 0.07771453711553666)

In [23]:
model = linear_model.Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=5000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False)
model.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=5000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [24]:
eval(model, X_test, y_test)

RMSLE: 0.4970
R2 value: -16.6025


In [29]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']
X_submit = scaler.transform(X_submit)

eval(model, X_submit, y_submit)

RMSLE: 0.4844
R2 value: -11.7495


In [32]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
pred_y = model.predict(X_submit)

submission = pd.DataFrame()
submission['MemberID'] = data['MemberID']

pred_y = pred_y.clip(min=0)
pred_y = pred_y.clip(max=15)
# pred_y = pred_y.astype(int)

submission['DIH'] = pred_y
submission.groupby(['DIH']).size()
submission.to_csv('../5.ensemble/lasso.csv',index=False)

In [31]:
submission.groupby(['DIH']).size()

DIH
0    62470
1     7587
2     1205
3      139
4       17
5       12
6        3
7        2
dtype: int64